### Quantum Fisher Information matrix

<img src="others\qfim.png">

I have coded QFIM function and test it on 1qubit case

$\partial_\theta|\psi\rangle$ can be calculated by on the below equation (from Le Bin Ho):

<img src="others/grad_psi.png" width=500px/>

The QNG are not simple as our imagin, the det of QFIM is 0 at almost step, so when using Moore-Penrose pseudo inverse the results're not good. The new solution is based on [this paper](https://arxiv.org/pdf/1909.02108.pdf). It seems to more complex, there are many that need to code:

- Create function that return the generator for every circuit

- Calculating the Fubini-Study tensor

- Divide circuit into some layer, and use recursion on it.

$R_X(\theta)=e^{i\frac{\theta}{2}X}$

In [1]:
import qiskit
import numpy as np
import qtm.base_qtm, qtm.constant, qtm.qtm_1qubit, qtm.qtm_nqubit
import matplotlib.pyplot as plt
from types import FunctionType

In [22]:
def create_circuit(qc, thetas):
    # |psi_0>: state preparation
    qc.ry(np.pi / 4, 0)
    qc.ry(np.pi / 3, 1)
    qc.ry(np.pi / 7, 2)
    # V0(theta0, theta1): Parametrized layer 0
    qc.rz(thetas[0], 0)
    qc.rz(thetas[1], 1)
    # W1: non-parametrized gates
    qc.cnot(0, 1)
    qc.cnot(1, 2)
    # V_1(theta2, theta3): Parametrized layer 1
    qc.ry(thetas[2], 1)
    qc.rx(thetas[3], 2)
    # W2: non-parametrized gates
    qc.cnot(0, 1)
    qc.cnot(1, 2)
    return qc

def create_psi():
    qc = qiskit.QuantumCircuit(3, 3)
    qc.ry(np.pi / 4, 0)
    qc.ry(np.pi / 3, 1)
    qc.ry(np.pi / 7, 2)
    return qc

generator_ry = np.array([
    [0, -1j],
    [1j, 0]
], dtype=np.complex128)

# calculate g_ij
qc = create_psi()
psi_qc = qiskit.quantum_info.Statevector.from_instruction(qc).data
psi_qc = np.expand_dims(psi_qc, 1)

g = np.zeros([2, 2])
g[0, 0] = (np.transpose(np.conjugate(psi_qc))).dot(generator_ry)

print(g)



ValueError: shapes (1,8) and (2,2) not aligned: 8 (dim 1) != 2 (dim 0)

In [7]:
qc = create_psi()
qc.measure(0, 0)
qc.measure(1, 1)
counts = qiskit.execute(qc, backend = qtm.constant.backend, shots = qtm.constant.num_shots).result().get_counts()
print(counts)

{'000': 5222, '001': 918, '010': 1771, '011': 281}


In [9]:
import pennylane as qml
from pennylane import numpy as np

dev = qml.device("default.qubit", wires=3)
g0 = np.zeros([2, 2])
params = np.array([0.432, -0.123, 0.543, 0.233])

def layer0_subcircuit(params):
    """This function contains all gates that
    precede parametrized layer 0"""
    qml.RY(np.pi / 4, wires=0)
    qml.RY(np.pi / 3, wires=1)
    qml.RY(np.pi / 7, wires=2)
@qml.qnode(dev)
def layer0_diag(params):
    layer0_subcircuit(params)
    return qml.var(qml.PauliZ(0)), qml.var(qml.PauliZ(1))


# calculate the diagonal terms
varK0, varK1 = layer0_diag(params)
g0[0, 0] = varK0 / 4
g0[1, 1] = varK1 / 4

print(g0)

[[0.125  0.    ]
 [0.     0.1875]]


In [21]:
dev = dev = qml.device("default.qubit", wires=2)
@qml.qnode(dev)
def A():
    qml.Hadamard(wires = 0)
    qml.Hadamard(wires = 1)
    return qml.probs(wires=[0])

print(A())

[0.5 0.5]
